In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(20)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.148.0.110:34496 Dashboard: http://10.148.0.110:8787/status,Cluster Workers: 36 Cores: 144 Memory: 1800.00 GB


---

# store grid

In [6]:
ds_index, ds = get_compressed_level_index(grid_dir)

ds = ds.drop(['dxG','dyG','dxC','dyC','rAw','rAs','rAz'])
ds = ds.drop(['hFacC','hFacW','hFacS'])
ds = ds.drop(['Z', 'Zp1', 'Zu', 'Zl', 'drC', 'drF','PHrefC','PHrefF'])
print(ds)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:721: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    YC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 432

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:235: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


In [7]:
out_dir = scratch+'/'
print(out_dir)

file_out = out_dir+'mit_grid.nc'
ds.to_netcdf(file_out, mode='w')


/home1/scratch/xyu/


In [8]:
dst = ds.transpose('face', 'i', 'j', 'i_g', 'j_g', 'k', 'k_l', 'k_p1', 'k_u')
print(dst)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, i, j) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    YC       (face, i, j) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 432

In [9]:
file_out = out_dir+'mit_grid_t.nc'
dst.to_netcdf(file_out, mode='w')

--- 

# automatic rewriting of all variables (for spectral calculations)

# transposed data: (i,j,time), 1 file per face, time windows

In [6]:
#face_60 = [1, 2, 4, 5, 7, 8, 10, 11]
face_60 = [1, 2, 4, 5]

print(face_60)

[1, 2, 4, 5]


In [7]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSV']

transpose = True # False untested

if transpose:
    Nt = 24*10*6 # time windows to consider
    out_dir = scratch+'/mit_nc_new/'
    fsize_bound = 15*1e9*6
else:
    Nt = 1
    out_dir = scratch+'/mit_nc/'    
    fsize_bound = 60*1e6    

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    #it = np.arange(time.size/Nt-1).astype(int)*Nt
    it = np.arange(2).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    #for face in range(ds['face'].size):    
    for face in face_60:
        for i, t in enumerate(it):
            #
            file_out = out_dir+'/%s_f%02d_t%02d.nc'%(v,face,i)
            if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
                dv = ds[v].isel(time=slice(t,t+Nt), face=face) 
                # should store grid data independantly in a single file
                dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
                #
                if transpose:
                    dv = dv.chunk({'time': dv['time'].size, 'i': 432, 'j': 432})
                    dv = dv.transpose('i','j','time')
                    chunksizes = [432, 432, dv['time'].size]
                else:
                    dv = dv.chunk({'i': 432, 'j': 432})
                    chunksizes = [1, 432, 432]
                #print(dv)
                #
                while True:
                    try:
                        %time dv.to_netcdf(file_out, mode='w', unlimited_dims=['time'], \
                                           encoding={'SSV': {'chunksizes': chunksizes}})
                    except:
                        print('Failure')
                    if os.path.isfile(file_out) and os.path.getsize(file_out)>fsize_bound:
                        #
                        print('face=%d / i=%d'%(face,i))
                        break
            else:
                print('face=%d / i=%d - allready processed'%(face,i))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler inst

CPU times: user 1min 28s, sys: 4.5 s, total: 1min 33s
Wall time: 13min 24s
face=1 / i=0


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 1min 28s, sys: 4.17 s, total: 1min 32s
Wall time: 12min 6s
face=1 / i=1


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


...

CPU times: user 1min 28s, sys: 4.48 s, total: 1min 33s
Wall time: 12min 7s
face=4 / i=1


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 1min 28s, sys: 4.09 s, total: 1min 32s
Wall time: 11min 55s
face=5 / i=0


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 1min 29s, sys: 4.5 s, total: 1min 33s
Wall time: 12min 3s
face=5 / i=1


--- 

# standard data layout: (face, j, i), 1 file per time (for movies)

In [ ]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSV']

Nt = 1
#out_dir = datawork+'/mit_nc/'
out_dir = scratch+'/mit_nc_t/'    
fsize_bound = 13*60*1e6

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    it = np.arange(time.size/Nt-1).astype(int)*Nt
    #it = np.arange(300).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    for i, t in enumerate(it):
        #
        file_out = out_dir+'/%s_t%04d.nc'%(v,i)
        if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
            dv = ds[v].isel(time=slice(t,t+Nt)) 
            # should store grid data independantly in a single file
            dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
            #
            #print(dv)
            #
            while True:
                try:
                    #print(dv)
                    %time dv.to_netcdf(file_out, mode='w')                    
                except:
                    print('Failure')
                if os.path.isfile(file_out) and os.path.getsize(file_out) > fsize_bound:
                    #
                    print('i=%d, iter=%d'%(i, iters[i].values))
                    break
        else:
            print('i=%d, iter=%d - allready processed'%(i, iters[i].values))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:721: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:235: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/worker.py:742: UserWarning: Large object of size 3.07 MB detected in task graph: 
  ("('astype-5bfade5531dee8ab8d957f6434906633', 

CPU times: user 368 ms, sys: 8 ms, total: 376 ms
Wall time: 10.7 s
i=0, iter=228096
CPU times: user 356 ms, sys: 8 ms, total: 364 ms
Wall time: 4.8 s
i=1, iter=228240
CPU times: user 352 ms, sys: 8 ms, total: 360 ms
Wall time: 4.94 s
i=2, iter=228384
CPU times: user 340 ms, sys: 24 ms, total: 364 ms
Wall time: 4.97 s
i=3, iter=228528
CPU times: user 344 ms, sys: 20 ms, total: 364 ms
Wall time: 4.96 s
i=4, iter=228672
CPU times: user 352 ms, sys: 12 ms, total: 364 ms
Wall time: 5.04 s
i=5, iter=228816
CPU times: user 360 ms, sys: 12 ms, total: 372 ms
Wall time: 4.89 s
i=6, iter=228960


In [8]:
cluster.scheduler.close()

<Future finished result=None>

In [16]:
client.restart()

Client Scheduler: tcp://10.148.0.76:8786 Dashboard: http://10.148.0.76:8787/status,Cluster Workers: 20 Cores: 280 Memory: 2000.00 GB


---

```
aponte/mit_tmp% ncdump -sh Eta_f01_4.nc

netcdf Eta_f01_4 {
dimensions:
	time = UNLIMITED ; // (24 currently)
	i = 4320 ;
	j = 4320 ;
variables:
	int64 i(i) ;
		i:standard_name = "x_grid_index" ;
		i:axis = "X" ;
		i:long_name = "x-dimension of the t grid" ;
		i:swap_dim = "XC" ;
		i:_Storage = "contiguous" ;
		i:_Endianness = "little" ;
	int64 j(j) ;
		j:standard_name = "y_grid_index" ;
		j:axis = "Y" ;
		j:long_name = "y-dimension of the t grid" ;
		j:swap_dim = "YC" ;
		j:_Storage = "contiguous" ;
		j:_Endianness = "little" ;
	int64 face ;
		face:standard_name = "face_index" ;
		face:_Endianness = "little" ;
	double time(time) ;
		time:_FillValue = NaN ;
		time:_Storage = "chunked" ;
		time:_ChunkSizes = 512 ;
	float Eta(i, j, time) ;
		Eta:_FillValue = NaNf ;
		Eta:coordinates = "face" ;
		Eta:_Storage = "chunked" ;
		Eta:_ChunkSizes = 432, 432, 24 ;

// global attributes:
		:_NCProperties = "version=1|netcdflibversion=4.6.1|hdf5libversion=1.10.1" ;
		:_Format = "netCDF-4" ;
}
```